
# 3D rendering single subject
Generates 3D rendering of all probe trajectories for a single subject.

The trajectory plotted are (pair):
- Channel locations based on the user picks (from histology)
- Linear Interpolation based on the picks
One color per pair.


In [ ]:
# Author: Olivier
# environment installation guide https://github.com/int-brain-lab/iblenv
# run "%qui qt" magic command from Ipython prompt for interactive mode
import numpy as np
from mayavi import mlab
from one.api import ONE
from iblutil.util import Bunch

import ibllib.plots
from atlaselectrophysiology import rendering
import ibllib.atlas as atlas

one = ONE(base_url='https://openalyx.internationalbrainlab.org')
subject = 'KS023'

ba = atlas.AllenAtlas(25)
channels_rest = one.alyx.rest('channels', 'list', subject=subject)
channels = Bunch({
    'atlas_id': np.array([ch['brain_region'] for ch in channels_rest]),
    'xyz': np.c_[np.array([ch['x'] for ch in channels_rest]),
                 np.array([ch['y'] for ch in channels_rest]),
                 np.array([ch['z'] for ch in channels_rest])] / 1e6,
    'axial_um': np.array([ch['axial'] for ch in channels_rest]),
    'lateral_um': np.array([ch['lateral'] for ch in channels_rest]),
    'trajectory_id': np.array([ch['trajectory_estimate'] for ch in channels_rest])
})

fig = rendering.figure()
for m, probe_id in enumerate(np.unique(channels['trajectory_id'])):
    traj_dict = one.alyx.rest('trajectories', 'read', id=probe_id)
    ses = traj_dict['session']
    label = (f"{ses['subject']}/{ses['start_time'][:10]}/"
             f"{str(ses['number']).zfill(3)}/{traj_dict['probe_name']}")
    print(label)

    color = ibllib.plots.color_cycle(m)
    it = np.where(channels['trajectory_id'] == probe_id)[0]
    xyz = channels['xyz'][it]
    ins = atlas.Insertion.from_track(xyz, brain_atlas=ba)

    mlapdv = ba.xyz2ccf(ins.xyz)
    # display the interpolated tracks
    mlab.plot3d(mlapdv[:, 1], mlapdv[:, 2], mlapdv[:, 0],
                line_width=3, color=color, tube_radius=20)
    # display the channels locations
    mlapdv_channels = ba.xyz2ccf(xyz)
    mlab.points3d(mlapdv_channels[:, 1], mlapdv_channels[:, 2], mlapdv_channels[:, 0],
                  color=color, scale_factor=50)
    # setup the labels at the top of the trajectories
    mlab.text3d(mlapdv[0, 1], mlapdv[0, 2], mlapdv[0, 0] - 500, label,
                line_width=4, color=tuple(color), figure=fig, scale=150)